In [ ]:
import torch 
from ipsl_dcpp.model.ipsl_dataset import IPSL_DCPP
from hydra import compose, initialize
from omegaconf import OmegaConf
import hydra
import numpy as np
with initialize(version_base=None, config_path="../conf"):
    cfg = compose(config_name="config")
train = IPSL_DCPP(
    'train',
    generate_statistics=True,
    lead_time_months=1,
    surface_variables=cfg.experiment.surface_variables,
    depth_variables=cfg.experiment.depth_variables,
    delta=False,
    normalization=''
)
train_dataloader = torch.utils.data.DataLoader(train,batch_size=10,shuffle=True,)



def z_normalize(data:np.ndarray,axes:tuple):
    mean = np.nanmean(data,axis=axes,keepdims=True)
    std_dev = np.nanstd(data,axis=axes,keepdims=True)
    # mean = np.broadcast_to(mean,data.shape)
    # std_dev = np.broadcast_to(std_dev,data.shape)
    # standardized_data = (data - mean) / std_dev
    return mean,std_dev

surface_means = []
#plev_means = []
depth_means =[]

surface_stds = []
#plev_stds = []
depth_stds = []

# datas = []
for count in range(100):
    # x,y,t1,t2 = batch
    batch = next(iter(train_dataloader))
  #  print(surface.shape)
    surface = batch['state_surface'].squeeze()
    #plev = batch['state_level'].squeeze()
    depth = batch['state_depth'].squeeze()
    # datas.append(x)
    print(count)
   # print(surface.shape)
  #  i_mean,i_std = z_normalize(surface.numpy(),(1,2))
    print(surface.shape)
    i_mean,i_std = z_normalize(surface.numpy(),(0))

   # t_mean,t_std = z_normalize(plev.numpy(),(1,2,3))
    #d_mean,d_std = z_normalize(depth.numpy(),(1,2,3))
    d_mean,d_std = z_normalize(depth.numpy(),(0,1))
    surface_means.append(i_mean)
    surface_stds.append(i_std)
   # plev_means.append(t_mean)
   # plev_stds.append(t_std)
    depth_means.append(d_mean)
    depth_stds.append(d_std)
print(len(surface_means))
surface_means_out = np.nanmean(np.stack(surface_means),axis=0)
surface_stds_out = np.nanmean(np.stack(surface_stds),axis=0)
#plev_means_out = np.nanmean(np.array(plev_means),axis=0)
#plev_stds_out = np.nanmean(np.array(plev_stds),axis=0)
depth_means_out = np.nanmean(np.stack(depth_means),axis=0)
depth_stds_out = np.nanmean(np.stack(depth_stds),axis=0)


In [9]:
import numpy as np

surf_mean = np.load('../data/spatial_multi_var_surface_means.npy')

In [12]:
surf_mean[0,50]

array([[222.13098, 222.13098, 222.13098, ..., 222.13098, 222.13098,
        222.13098],
       [225.41875, 225.3069 , 225.20682, ..., 225.64653, 225.54465,
        225.5196 ],
       [225.86147, 225.67654, 225.50385, ..., 226.36176, 226.21217,
        226.02869],
       ...,
       [259.12234, 259.20468, 259.25854, ..., 258.92667, 259.02786,
        259.08823],
       [259.04004, 259.0749 , 259.1043 , ..., 258.96646, 258.9745 ,
        258.98495],
       [258.30215, 258.30215, 258.30215, ..., 258.30215, 258.30215,
        258.30215]], dtype=float32)